<a href="https://colab.research.google.com/github/LeeEdzl/Colabwork/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

교차 검증과 그리드 서치

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


훈련 세트와 검증 세트의 평가 점수를 이용하여 최적화
<p>
최적화 완료 후 테스트 세트로 모델을 최종 평가

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


교차검증

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target) #모델 객체, train_input, train_target
print(scores)

{'fit_time': array([0.02516508, 0.02476549, 0.02084184, 0.01790047, 0.01754546]), 'score_time': array([0.00187874, 0.00163555, 0.00167108, 0.00896788, 0.00891447]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


test_score이지만 테스트 점수가 아닌 검증 점수임

In [ ]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(StratifiedKFold())
print(scores)
print(np.mean(scores['test_score']))

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
{'fit_time': array([0.01494122, 0.00729203, 0.00745964, 0.00731015, 0.00742936]), 'score_time': array([0.00144744, 0.0009706 , 0.00091481, 0.00099182, 0.0010469 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(scores)
print(np.mean(scores['test_score']))

{'fit_time': array([0.01715565, 0.02950501, 0.02440882, 0.02857733, 0.01153231,
       0.02629995, 0.02750659, 0.01204133, 0.0207243 , 0.04852009]), 'score_time': array([0.00948763, 0.00162005, 0.00167561, 0.00159311, 0.00139332,
       0.02484608, 0.00154209, 0.00130844, 0.00851536, 0.00165105]), 'test_score': array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
       0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])}
0.8574181117533719


<h2>하이퍼파라미터 튜닝

그리드서치
<p>
하이퍼파리미터 탐색과 교차 검증을 한번에 수행함
<p>cross_validate()함수를 호출할 필요가 없음

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
print(gs)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})


In [ ]:
#n_jobs는 코어 수 (병렬 처리가 가능하기 때문에 사용함)
#무조건 n_jobs=-1로 사용하면 됨

In [ ]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
#객체 생성(조건 생성) -> 훈련 -> 최적의 매개변수 조합을 찾아줘야 함
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_estimator_)
print(gs.best_params_)

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


그리드 서치를 이용한 최적화 과정 정리
<p>
1. 탐색할 매개변수 지정
<p>
2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음
<p>
3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아닌) 전체 훈련 세트를 사용해 최종 모델을 훈련함.


In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


랜덤서치

In [ ]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([3, 4, 6, 8, 2, 1, 7, 6, 0, 5])

In [ ]:
#10번의 샘플링 만으로는 고르게 샘플링이 되지 않기에 샘플링 숫자를 늘려 확인해봄

np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98, 106, 103, 106, 109,  72,  76, 133, 103,  94]))

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.86059757, 0.4952509 , 0.43246632, 0.86707681, 0.82988615,
       0.9213557 , 0.02944704, 0.15686403, 0.99120076, 0.9322644 ])

결정트리 모델에서 최적값 찾기

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
dt = gs.best_estimator_

print(dt)
print(dt.score(test_input, test_target))

DecisionTreeClassifier(max_depth=39,
                       min_impurity_decrease=0.00034102546602601173,
                       min_samples_leaf=7, min_samples_split=13,
                       random_state=42)
0.86


In [ ]:
#그리드 서치와 랜덤 서치를 이용하여 최적화..?

In [ ]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': range(5, 20),
                                        'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                                        'min_samples_split': range(2, 100, 10)},
                   random_state=42)

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'min_samples_split': 32, 'min_impurity_decrease': 0.0001, 'max_depth': 15}
0.8535701858295699
0.8353846153846154
